In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
# document = Document("Preprocessing/208-230.docx")
# document = Document("Converted/pdf2go-execution.docx")
document = Document("RPL/A/3. Preprocessed/[2]Dictionary IND-ENG-323-345.docx")

In [3]:
# for paragraph in document.paragraphs:
    
#     len(paragraph.runs)
    
# #     for run in paragraph.runs :
# #         print(run.text)
    
    
#     # first line indent (segitiga atas di ruler)
#     first = paragraph.paragraph_format.first_line_indent
    
#     # left indent (segitiga bawah di ruler)
#     left = paragraph.paragraph_format.left_indent
    
#     print(first,left,paragraph.text+"\n")

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# Perisiapan Word :
# - ubah panah jadi >
# - –/~ yang reguler dan bold dijadikan italic
# - ubah font, kemudian tanda yang tidak terbaca seperti di kata "major" (-^w) diganti menjadi <
# - tanda yang tidak terbaca seperti di kata "brother-in-law" (-) diganti menjadi ^w
# - tanda "- " diganti menjadi "<", contoh me- ngelabui (PERTIMBANGKAN LAGI)
# - ubah tanda "..." menjadi "_" (garis bawah)
# - s.t. = something
# - s.o.'s = someone's
# - s.o. = someone
# - q.v. = which see (refer to that word)
# - k.o. = kind(s) of
# - o.s. = oneself
# - e.o. = eachother
# - replace .^w jadi .^p
# - menghapus setiap header halaman dan garisnya



In [6]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

#         word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w) or word == "/" or w == "." or w == "(")]
        
        #update 1
        word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w))]
    
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
        #update 3
#         if word in roman_numerals or re.search("\d", word) or word == "/" or word == "." or word == "(":
        
        if word in roman_numerals or re.search("\d", word) :
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("/\s","/",x[idx])
        x[idx] = re.sub("\snya","nya",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])
#         x[idx] = re.sub("\(=","(",x[idx])
        
#         x[idx] = re.sub("s\.t\.|s\.t","something",x[idx])
#         x[idx] = re.sub("s\.o\.'s","someone's",x[idx])
#         x[idx] = re.sub("s\.o\.","someone",x[idx])
#         x[idx] = re.sub("q\.v\.","which see (refer to that word)",x[idx])
#         x[idx] = re.sub("k\.o\.","kind(s) of",x[idx])
#         x[idx] = re.sub("o\.s\.","oneself",x[idx])
#         x[idx] = re.sub("e\.o\.","eachother",x[idx])
#         x[idx] = re.sub(";|;\s","",x[idx])

        x[idx] = re.sub("\*\*","e.g.",x[idx])
        x[idx] = re.sub("\*","i.e.,",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
#         x[idx] = re.sub("_","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :        
        x[4] = re.split(",|;",x[4])
        listPanah = []
        
        for idx in range(len(x[4])):
            if '(' in x[4][idx] :
                x[4][idx], y = re.sub(r"[/(].*?[/))=]","",x[4][idx]),re.sub(r"[()=]","",x[4][idx])
                x[4][idx], y = re.sub(r"\s$","",x[4][idx]),re.sub(r"\s$","",y)
                x[4][idx] = re.sub("\s{2}|\s{3}"," ",x[4][idx])
#                 x[4].append(x)
                listPanah.append(y)
        
        if len(listPanah) != 0 :
            for val in listPanah :
                x[4].append(val)
    
    #split terjemahan
    if x[9] != "" :
        x[9] = re.split(",|;",x[9])
        x[9] = replaceData(x[9])
        
        if "" in x[9] :
            x[9].remove("")
        
        for idx in range(len(x[9])):
            x[9][idx] = re.sub("_","...",x[9][idx])
            
        if len(x[9]) < 1 :
            x[9] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" or word == "cp":
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text):

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            #update 2
            
#             try :
            if (index+1 < len(paragraph.runs) and paragraph.runs[index+1].italic) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word
#             except :
#                 if terjemahan == "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = word
#                 elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = terjemahan+" "+word

        # update sementara
        try :
            if re.search("\)", word) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        except :
            if re.search("\)", word) :
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        
#         keterangan = "satu"

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
#         keterangan = "dua"

    else :
        tempGabungan.append(word)
#         keterangan = "tiga"
    
    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [7]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
# list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod","neo"]
list_data = ["something", "someone's", "someone", "which see (refer to that word)", "kind(s) of", "oneself", "each other","something ", "someone's ", "someone ", "which see (refer to that word) ", "kind(s) of ", "oneself ", "each other "]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
perbandingan = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
boolSimilar = False
tempLema = ""
tempSublema = ""
# tempLemaSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         print("word = "+run.text)
#         print("boolSimilar = "+str(boolSimilar))
        
#         try :
#             print("prev = "+paragraph.runs[index-1].text)
#         except :
#             print("prev = -")
#         print("index = "+str(index))
        
#         print("========================")
#         print("length = "+str(len(paragraph.runs)))
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600 
        
        #deteksi kata bold
        if run.bold and run.italic == None  :
            if run.text.isspace() == False :
                #baca indent lema
                if (((first == None or first == 0 or first == 76200) and (left <= 76835)) or ((first == -152400 or first == -153035) and left >= 227965)) :      

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                        print(x[0])
                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    boolSimilar = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
#                         print("run.text = "+run.text)
#                         print(word)
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
                        if boolLemaSublema == False :

                            results = cleanParenthesis(word)

                            lema = results[0]
                            similar = results[1]
#                             tempLemaSublema = results[2]
                            boolLemaSublema = [3]
                            
                            tempLema = lema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) :
#                                 print(word)
                                similar = word

                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :

                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :
                            
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                results = cleanParenthesis(word)

                                sublema = results[0]
                                similar = results[1]
#                                 tempLemaSublema = results[2]
                                tempSublema = results[2]
                                boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) :
                                    similar = word

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
#             print("sublema = "+sublema)
#             print("similar = "+similar)
#             print("tempLemaSublema = "+tempLemaSublema)
            
        elif (run.italic and run.bold) or run.italic :
            if run.text.isspace() == False :
                
#                 print("====================")
#                 print("word = "+run.text)
#                 print("prev = "+paragraph.runs[index-1].text)
                
#                 try :
#                     print("prev-2 = "+paragraph.runs[index-2].text)
#                 except :
#                     print("prev-2 tdk ada")
                
# #                 print(index)
# #                 print(index > 0)
# #                 print(paragraph.runs[index-1].text in list_data)
#                 print("boolSimilar = "+str(boolSimilar))
#                 print((index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and))

                if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) :
                    boolSimilar = True
    
                if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) or ((index > 0 and re.search("_|;", paragraph.runs[index-1].text)) or (index > 1 and re.search("_|;", paragraph.runs[index-2].text))) :
#                 if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) or ((index > 0 and re.search("_", paragraph.runs[index-1].text) and re.search("–|~", run.text)) or (index > 1 and re.search("_", paragraph.runs[index-2].text)) and re.search("–|~", run.text)) :
#                 if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) :
                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                        print(x[0])
                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""
                            
                    boolPanah = False
                    boolSimilar = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []
                    
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]
                    
#                     print(results)
                    
                elif boolPanah == False and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :
                    word = re.sub("\.","", run.text)
                    word = cleanWord(word)
                    
#                     print(word)

                    if word != None :
                        panah = word
                        boolPanah = True
                        word = ""

                else :
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]

#                     print(results)
#                 print("========================")
                
        # deteksi kata reguler (akronim, terjemahan)
        else : 
            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
#                     if "–" in tempGabungan or "~" in tempGabungan:
#                         gabungan = " ".join(tempGabungan)
#                         gabungan = re.sub("–|~",tempLemaSublema,gabungan)
#                         gabungan = re.sub("  "," ", gabungan)

#                         results = cleanParenthesis(gabungan)

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

#                     print(tempGabungan)

                    if "–" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("–",tempLema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        if (boolSimilar != True) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []
                    
                    elif "~" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("~",tempSublema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        if (boolSimilar != True) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    else :
                        if len(tempGabungan) != 0:
                            
#                             print ("length = "+str(len(tempGabungan)))
#                             print ("index = "+str(index))
#                             print(tempGabungan)
#                             print("tempLema = "+tempLema )

                            temp = " ".join(tempGabungan)
    
#                             print("temp = "+temp)
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
#                                 temp = " ".join(tempGabungan)
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempLemaSublema.lower() in temp.lower()) :
                            elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempSublema.lower() in temp.lower()) :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                if (boolSimilar != True) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []
                            
                            elif index > 1 and len(tempGabungan) <= 3:
#                                 temp = " ".join(tempGabungan)

                                if (boolSimilar != True) :
                                    terjemahan = terjemahan+" "+temp
                                else :
                                    similar = temp
                                
                                boolSimilar = False
                                temp = ""
                                tempGabungan = []
            
#                                 print("hahaha")

                            else :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                if (boolSimilar != True) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []

                #AKRONIM
                # mengambil akronim
                if (index > 0 and re.search("\[", paragraph.runs[index-1].text)) and re.search("and", word):
                    tempAkronim.append(word)

                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

#                     try :
                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
#                     except :
#                         if re.search("\]", paragraph.runs[index].text) :
#                             akronim = " ".join(tempAkronim)
#                             akronim = re.sub(r'  ',' ', akronim)
#                             tempAkronim = []
                else :
                    #PANAH   
                    
#                     print("prev-1 = "+paragraph.runs[index-1].text)
                    
#                     if index > 1 :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
                        
#                     print("word = "+word)
#                     print("terjemahan = "+terjemahan)
#                     print("index = "+str(index))
#                     print("length = "+str(len(paragraph.runs)))
                    
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)
                     
#                         print("word 2= "+word)
#                         print(tempWord)

                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                            
#                         print("panah 2 = "+panah)
                    
                    if boolPanah == True and re.search("\.", word) :
#                         print("word 3= "+word)
                        
                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""
#                         print("panah 3 = "+panah)
                    
                    
#                     if word != "" and word != "." and (paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">")) :
                    if word != "" and word != "." and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :

#                         print("word 1= "+word)

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                         print("panah = "+panah)
                        
#                         print("panah 1 = "+panah)
#                             
#                             try :
#                                 print(paragraph.runs[index+1].text)
#                                 print(panah)
#                             except :
#                                 print(paragraph.runs[index+0].text)
#                                 print(panah)

                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "opp") or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word
                
                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "cp") or (index > 1 and paragraph.runs[index-2].text == "cp")):
                        re.sub("\s","",word)

                        perbandingan = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if re.search("and", word) and ((index > 0 and (paragraph.runs[index-1].bold) == True or paragraph.runs[index-1].italic) == True ) and ((index+1 < len(paragraph.runs) and (paragraph.runs[index+1].bold == True or paragraph.runs[index+1].italic == True)) or (index+2 < len(paragraph.runs) and (paragraph.runs[index+2].bold == True or paragraph.runs[index+2].italic == True))):
                            word = ""

                        if word == "[" or word == "]" or re.search(">", word) :
                            word = ""

#                         if terjemahan == "" and word not in roman_numerals and boolPanah != True :
                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
#                         elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print("prev = "+paragraph.runs[index-1].text)
#                         print("word = "+word)
#                         print("terjemahan = "+terjemahan)
#                         print("run.text = "+run.text)
#                         if (re.search("\.|\!|\?", run.text) and ( index < len(paragraph.runs)-1 and paragraph.runs[index+1] == " " )) or re.match("\w\)", run.text) :
                        if (re.search("\.|\!|\?", run.text)) or re.match("\w\)", run.text) :
        
#                             print("hihi")
            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            perbandingan = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            boolSimilar = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
                            
#                             boolPanah = False
        index += 1                 

['', '', '', '', '', '', '', '', '', ['323']]
['gigit', '', '', '', '', '', '', '', '', '']
['', 'getuk', '', '', ['GETAK getuk'], '', '', '', '', '']
['', 'getun', '', '', ['GEGETUN'], '', '', '', 'J', '']
['', 'geuchik', '', '', '', '', '', '', 'Ac', ['village head']]
['geudeu-geudeu', '', '', '', '', '', '', '', 'Ac', ['Acehnese wrestling']]
['geulis', '', '', '', '', '', '', '', 'S', ['beautiful']]
['géwang', '', '', '', ['GIWANG'], '', '', '', '', ['motherofpearl']]
['géwang', '', '', '', ['GIWANG'], '', '', '', '', ['ear stud']]
['géwang', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['GEBANG'], '', '', '', '', ['gebang palm', 'Corypha elata']]
['gewésten', '', '', '', '', '', '', '', 'D col', ['regions', 'small territorial units']]
['géyat-géyot', '', '', '', '', '', '', '', '', ['to dance around']]
['géyong', '', '', '', '', '', '', '', 'J', ['thickset woman with a large behind']]
['géyong', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'Jv', ['to

['', '', '', '', '', '', '', '', '', ['– satu first gear']]
['', '', 'masuk gidu dua', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['in second gear']]
['', '', '', '', '', '', '', '', '', ['persnéling transmission gear']]
['', '', '', '', '', '', '', '', '', ['edge']]
['', '', '', '', '', '', '', '', '', ['– hutan edge of the for est']]
['', '', '', '', '', '', '', '', '', ['– tanggal rawan murah to get a chance when it’s too late']]
['', '', '', '', '', '', '', '', '', ['lunak – daripada lidah very gentle/mild']]
['', '', 'dan lidah adaka-lanya bergigit juga', '', '', '', '', '', '', ['– husbands and wives or siblings are sometimes also at odds with e.o']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['– air']]
['', '', '', '', '', '', '', '', '', ['the rippled surface of water']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['the edg e o f th e se a (o n th e beach)', 'sea line', 'horizon']]
['', '

['', '', 'dalam hawa dingin', '', '', '', '', '', '', ['~ in biting cold']]
['', '', 'anjing yang menyalak tiada', '', '', '', '', '', '', ['~ his bark is worse than his bite']]
['', '', 'belum bergigi sudah ingin menggigit', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['has no power but wants to take action']]
['', 'menggigiti', '', '', '', '', '', '', 'pl obj', ['to bite', 'etc']]
['gigolo', '', '', '', '', '', '', '', '', ['324']]
['', 'tergigit', '', '', '', '', '', '', '', '']
['', '', '', 'kegigit', '', '', '', '', 'coq', '']
['', '', '', '', '', '', '', '', '', ['bitten']]
['', '', '', '', '', '', '', '', '', ['bitten unintentionally']]
['', '', '', '', '', '', '', '', '', ['~ lidah unable to take action because of a debt of gratitude to s.o']]
['', '', '~ oléhku lidahku ketika sedang makan tadi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['I accidentally bit my tongue when I was eating']]
['', '', '', '', '', '', '', '', '', ['taken i

['', 'menggilas', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to pulverize', 'crush', 'grind', 'to roll', 'flatten']]
['', '', 'cabé di batu cobékan', '', '', '', '', '', '', ['~ to grind hot peppers in a mortar']]
['', '', '', '', '', '', '', '', '', ['to run over (s.o./a']]
['', '', '', '', '', '', '', '', '', ['dog', 'etc.)']]
['', '', '', '', '', '', '', '', '', ['knock down', 'run into s.o']]
['', '', '', '', '', '', '', '', '', ['collide with (an other car)']]
['', '', 'Kerbau itu mati digilas keréta api', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['The water buf falo was run over and killed by a train']]
['', '', '', '', '', '', '', '', '', ['to stifle', 'suppress', 'silence (opposition)']]
['', '', 'Apakah yang dapat kulakukan untuk ~ semua perasaan?', '', '', '', '', '', '', ['What can I do to suppress all my feelings?']]
['', 'menggilaskan', '', '', '', '', '', '', '', ['to run down/over']]
['', 'tergilas', '', '', '', '', '',

['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['The head of the RT had the members of the RT take turns doing the rounds']]
['', 'mempergilirkan', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to have s.o']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['do s.t']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['alternately', 'have s.o']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['take a turn']]
['', '', '', '', '', '', '', '', '', ['to have (s.o.)']]
['', '', '', '', '', '', '', '', '', ['take a turn with', 'turn s.t']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['over (to)']]
['', '', '', 'Tongkat komando dipergilirkan ke Angkatan Udara', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['Command was turned over to the Air Force']]
['', 'giliran', '', '', '', '', '', '', '', '']
[''

...

['', '', '', '', '', '', '', '', '', ['susunan ~ judicial organization']]
['', '', 'kehakiman militér', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['military justice']]
['', 'penghakiman', '', '', '', '', '', '', '', ['punishment']]
['hakimah', '', '', '', '', '', '', '', 'A', ['wise (woman)']]
['hakir', '', '', '', '', '', '', '', 'A', ['mean or degraded person']]
['hakulyakin', '', '', '', '', '', '', '', 'A', ['absolute truth']]
['hal', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'A', '']
['', '', '', '', '', '', '', '', '', ['problem', 'matter', 'case', 'cause', 'reason']]
['', '', '', '', '', '', '', '', '', ['situation', 'inci-dent', 'affair']]
['', '', '', '', '', '', '', '', '', ['concerning', 'regarding', 'about', 'in the matter of']]
['', '', '', '', '', '', '', '', '', ['ada – there’s s.t']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['going on']]
['', '', 'dalam hal', '', '', '', '', '', '', '

['', '', '', '', '', '', '', '', '', ['– muka front page']]
['', '', '', '', '', '', '', '', '', ['– putih blank page']]
['', '', '', '', '', '', '', '', '', ['– tengah centerfold']]
['', 'berhalaman', '', '', '', '', '', '', '', ['with/have pages']]
['', '', '', '', '', '', '', '', 'cla', ['destar ~ k.o']]
['', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['full turban']]
['', 'berhalaman-halaman', '', '', '', '', '', '', '', ['pages and pages']]
['halamanisme', '', '', '', '', '', '', '', '', ['kampung ~ provincialism']]
['halang', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['ALANG'], '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['weir']]
['', '', '', '', '', '', '', '', '', ['interdiction']]
['', '', '', '', '', '', '', '', '', ['– rintang steeplechase']]
['', 'menghalang', '', '', '', '', '', '', '', ['to obstruct', 'impede', 'hinder', 'prevent', 'block']]
['', '', '', '', '', '', '', '', '', ['~ pan danganku to block my view']]
[''

['halir', '', '', '', ['ALIR'], '', '', '', '', '']
['halkah', '', '', '', '', '', '', '', 'A', '']
['', '', '', '', '', '', '', '', '', ['(ankle) bracelet', 'anklet']]
['', '', '', '', '', '', '', '', '', ['link']]
['', '', '', '', '', '', '', '', 'naut', ['– berséngkang stud link']]
['', '', '', '', '', '', '', '', '', ['Islamic study circle']]
['halkum', '', '', '', '', '', '', '', 'A', '']
['', '', '', '', '', '', '', '', '', ['Adam’s apple', 'larynx']]
['', '', '', '', ['LEKUM'], '', '', '', '', ['throat']]
['halma', '', '', '', '', '', '', '', 'D', ['Chinese checkers']]
['halo', '', '', '', '', '', '', '', 'D', ['hello']]
['', '', '', '', '', '', '', '', '', ['–', 'Sutomo di sini! sambutnya di pesawat télepon']]
['', '', '', '', '', '', '', '', '', ['“Hello', 'Sutomo speaking', '” he said on the phone']]
['', 'halo-halo', '', '', '', '', '', '', 'coq', ['microphone']]
['', 'berhalo-halo', '', '', '', '', '', '', '', ['to talk on the phone']]
['', '', 'tanp a éngko l', '', ['TÉLEP

['', '', '', '', '', '', '', '', '', ['– telinga musical treat']]
['ham', '', '', '', '', '', '', '', 'D', ['ham (meat)']]
['HAM', '', '', '', ['HAK asasi manusia'], '', '', '', 'init', '']
['ham', '', '', '', ['HÉm'], '', '', '', '', '']
['hama', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['AMA'], '', '', '', '', ['the natural enemies of a crop', 'plant disease', 'pest']]
['', '', 'ilmu', '', '', '', '', '', '', ['– entomology']]
['', '', '', '', '', '', '', '', '', ['plague', 'scourge']]
['–', '', '', '', '', '', '', '', '', ['microbes', 'germs (of a disease)', 'bacteria']]
['', '', '', '', '', '', '', '', '', ['kena – infected']]
['', '', '', '', '', '', '', '', '', ['suci – aseptic', 'ster-ilized']]
['', '', '', '', '', '', '', '', '', ['– bawang a rice pest', 'gall midge', 'Orseolia oryzae']]
['', '', '– beluk', '', '', '', '', '', '', '']
['', '', '', '', ['BELUK'], '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['– bodas pest which attacks vegetables', 'onion

['', '', '', '', '', '', '', '', '', ['a slave to illegal drugs']]
['', 'penghambaan', '', '', '', '', '', '', '', ['enslaving']]
['', 'perhambaan', '', '', '', '', '', '', '', ['servitude', 'subservience', 'slavery', 'bondage']]
['hamba', '', '', '', '', '', '', '', '', '']
['', '', 'burung hamba kera', '', '', '', '', '', '', '']
['', '', '', '', ['ANTING-ANTING'], '', '', '', '', ['large racquet-tailed drongo', 'Surniculus lugrubis']]
['hambal', '', '', '', ['AMBAL'], '', '', '', '', ['rug']]
['hambar', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['CEMPLANG', ' TAWAR'], '', '', '', '', ['tasteless', 'insipid', 'flat']]
['', '', '', '', '', '', '', '', '', ['Masa kannya – rasanya']]
['', '', '', '', '', '', '', '', '', ['Her cooking is tasteless']]
['', '', '', '', '', '', '', '', '', ['cold', 'indifferent', 'spiritless', 'vapid']]
['', '', 'Terhadap suaminya ia mengambil sikap yang – saja', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['She was cold and 

In [8]:
# lexicon.to_excel (r'A.xlsx', index = False, header=True)

In [9]:
re.search("\.\”|\.|\!|\?", ".” A")

<_sre.SRE_Match object; span=(0, 2), match='.”'>